# Data Cleaning

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
import warnings
import nltk
import spacy
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from textblob import TextBlob, Word
import pattern
from pattern.en import lemma, lexeme
from sklearn.feature_extraction import text 

warnings.filterwarnings("ignore")

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\joann\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1. Import Data

In [4]:
data_dir = 'C:\\Users\\joann\\OneDrive\\Desktop\\NUSWhispers Scrapper\\data'
pickle_dir = 'C:\\Users\\joann\\OneDrive\\Desktop\\NUSWhispers Scrapper\\pickle'

In [5]:
data = pd.read_csv(data_dir + "\\full_data.csv")

## 2. Clean Text Data

In [6]:
def clean_text(x):
    name = "-{}:".format(x["Reference"])
    clean = x["Content"].split(name)[0]
    return clean

In [7]:
data["Content_Clean"] = data.apply(clean_text, axis=1)

In [8]:
def clean_text_1(x):
    text = x["Content_Clean"]
    text = text.lower() # lowercase
    text = text.replace(".", " ")
    text = re.sub(r'[^\w\s]','', text)# remove punctuation
    text = ''.join(i for i in text if not i.isdigit()) # remove digits
    text = text.replace("“", "")
    text = text.replace("’", "")
    text = text.replace("”", "")
    text = ' '.join(text.split())

    return text

In [9]:
data["Content_Clean_1"] = data.apply(clean_text_1, axis=1)

### Stopwords

In [10]:
add_stop_words = ["just", "really", "year", "many", 
                  "like", "think", "want", "need", "say", "know", "time",
                   "tell", "thing", "say", "make", "come", "ask", "feel"]

In [11]:
stopwords_list = text.ENGLISH_STOP_WORDS.union(add_stop_words)

### Lemmatize

https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [12]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [13]:
def wordnet_lemmatizer(x):
    words_list = x["Content_Clean_1"].split(" ")
    lemmatizer = WordNetLemmatizer()
    results = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words_list]
    filtered = [word for word in results if word not in stopwords_list]
    
    output = " ".join(filtered)
    return output

In [14]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [15]:
def spacy_lemmatizer(x):
    doc = nlp(x["Content_Clean_1"])
    result = [token.lemma_ for token in doc]
    filtered = [word for word in result if word not in stopwords_list]
    output = " ".join(filtered)
    output = output.replace("-PRON-", "")
    output = " ".join(output.split())
    return output

In [16]:
def textblob_lemmatizer(x):
    sent = TextBlob(x["Content_Clean_1"])
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    filtered = [word for word in lemmatized_list if word not in stopwords_list]
    
    output = " ".join(filtered)
    return output

In [17]:
def pattern_lemmatizer(x):
    sentence = x["Content_Clean_1"]
    result = [lemma(word) for word in sentence.split()]
    filtered = [word for word in result if word not in stopwords_list]
    output = " ".join(filtered)
    
    return output

In [19]:
data["Spacy_Lemmatize"] =  data.apply(spacy_lemmatizer, axis=1)

In [21]:
main_data = data.loc[:,['Reference', 'Date_Clean', 'Content', 'Spacy_Lemmatize','Comment', 'Share', 'Angry',
       'Care', 'Haha', 'Like', 'Love', 'Sad', 'Wow', 'No Reactions',
       'No Category', 'Advice', 'Ask Prof Ben', 'Funny', 'Lost and Found',
       'Nostalgia', 'Rant', 'Romance']]

### Concat Category

In [ ]:
subset = main_data.loc[:, 'No Category':'Romance']

In [ ]:
final_category = []
for i in range(0, len(subset)):
    temp_category = []
    for col in subset.columns:
        if (subset[col][i] == 1):
            temp_category.append(col)
    final_category.append(temp_category)
            

In [ ]:
main_data["Category"] = final_category

In [ ]:
main_data['Category'] = main_data['Category'].apply(lambda x: ", ".join(set(x)))

### Total Reactions

In [ ]:
main_data["Total Reactions"] = main_data[["Angry", "Care", "Haha", "Like", "Love", "Sad", "Wow"]].sum(axis=1)

In [22]:
main_data.to_pickle(pickle_dir + "\\main_data.pkl")

## 3. Organizing The Data

In [170]:
advice_cat = data[data["Advice"] == 1]
prof_ben_cat = data[data["Ask Prof Ben"] == 1]
funny_cat = data[data["Funny"] == 1]
lost_found_cat = data[data["Lost and Found"] == 1]
nostalgia_cat = data[data["Nostalgia"] == 1]
rant_cat = data[data["Rant"] == 1]
romance_cat = data[data["Romance"] == 1]

In [171]:
advice_cat["Category"] = 'Advice'
prof_ben_cat["Category"] = 'Ask Prof Ben'
funny_cat["Category"] = 'Funny'
lost_found_cat["Category"] = 'Lost and Found'
nostalgia_cat["Category"] = 'Nostalgia'
rant_cat["Category"] = 'Rant'
romance_cat["Category"] = 'Romance'

In [172]:
column = "Spacy_Lemmatize"

In [173]:
advice_cat_clean = advice_cat.loc[:,["Category", column]]
prof_ben_cat_clean = prof_ben_cat.loc[:,["Category", column]]
funny_cat_clean = funny_cat.loc[:,["Category", column]]
lost_found_cat_clean = lost_found_cat.loc[:,["Category", column]]
nostalgia_cat_clean = nostalgia_cat.loc[:,["Category", column]]
rant_cat_clean = rant_cat.loc[:,["Category", column]]
romance_cat_clean = romance_cat.loc[:,["Category", column]]

In [174]:
df_cat = pd.concat([advice_cat_clean, prof_ben_cat_clean], axis=0)
df_cat = pd.concat([df_cat, funny_cat_clean], axis=0)
df_cat = pd.concat([df_cat, lost_found_cat_clean], axis=0)
df_cat = pd.concat([df_cat, nostalgia_cat_clean], axis=0)
df_cat = pd.concat([df_cat, rant_cat_clean], axis=0)
df_cat = pd.concat([df_cat, romance_cat_clean], axis=0)
df_cat = df_cat.reset_index().drop("index", axis=1)

In [175]:
df_cat = df_cat.groupby("Category")[column].apply(' '.join).reset_index().set_index('Category')

In [176]:
df_cat.to_pickle(pickle_dir + "\\corpus.pkl")

## Document Term Matrix

In [177]:
cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(df_cat[column])
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = df_cat.index
data_dtm

aa  aaa  aand  aback  abandon  abandonment  ability  \
Category                                                              
Advice           0    0     1      1        4            1        8   
Ask Prof Ben     0    0     0      0        0            0        0   
Funny            0    0     0      0        1            0        1   
Lost and Found   0    0     0      0        0            0        0   
Nostalgia        0    0     0      0        0            0        0   
Rant             1    1     1      1        2            0        5   
Romance          0    0     0      1        0            1        3   

                abilityaptitude  abit  abitmaybe  able  abnormally  abolish  \
Category                                                                      
Advice                        0    13          1    98           1        0   
Ask Prof Ben                  1     1          0    10           0        0   
Funny                         0     0          0     3           0        0   
Lost and Found                0     0          0     1           0        0   
Nostalgia                     0     0          0     6           0        0   
Rant                          0     6          0    53           1        1   
Romance                       0     4          0    33           1        0   

                abort  abortion  aboutplanne  aboutthere  abrasive  abruptly  \
Category                                                                       
Advice              4         6            1           1         1         2   
Ask Prof Ben        0         0            0           0         0         0   
Funny               0         0            0           0         0         0   
Lost and Found      0         0            0           0         0         0   
Nostalgia           0         0            0           0         0         0   
Rant                3         8            1           0         0         1   
Romance             0         1            0           0         1         1   

                absence  absent  absolute  absolutely  absorb  abstract  \
Category                                                                  
Advice                2       1         2          12       1         0   
Ask Prof Ben          0       0         0           0       0         1   
Funny                 0       0         0           0       0         0   
Lost and Found        0       0         0           0       0         0   
Nostalgia             0       0         0           0       0         0   
Rant                  1       1         3          12       0         0   
Romance               0       0         1           3       1         0   

                abstraction  absurd  abt  abundance  abundantly  abuse  \
Category                                                                 
Advice                    2       0    8          2           1     15   
Ask Prof Ben              2       0    0          1           0      0   
Funny                     1       0    0          0           0      0   
Lost and Found            0       0    0          0           0      0   
Nostalgia                 0       0    0          1           0      0   
Rant                      1       1    2          2           1      8   
Romance                   0       0    3          0           0      4   

                abusive  abysmal  abyss  acad  academia  academic  \
Category                                                            
Advice               15        1      1     0         1        15   
Ask Prof Ben          0        1      1     0         0         5   
Funny                 0        0      0     0         0         0   
Lost and Found        0        0      0     0         0         0   
Nostalgia             0        0      0     0         0         0   
Rant                  5        1      1     2         0        14   
Romance               6        0      1     0   

In [178]:
data_dtm.to_pickle(pickle_dir + "\\dtm.pkl")